# Régression Logistique

## 1. Importez les bibliothéques necessaires 

In [23]:
import pandas as pd

## 2. Importez le jeu de données 'weatherAUS.csv'

Ce jeu de données contient des observations météorologiques quotidiennes de nombreuses stations météorologiques australiennes.

Pour plus d'information sur le jeu de données: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [24]:
path='./Data_logit/weatherAUS.csv'
df = pd.read_csv(path)

## 3. Exploration des données

### Dimension de du jeu de données

In [25]:
df.shape


(142193, 24)

### Prévisulaiser le jeu de données

In [26]:
df.head()


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


### Afficher la liste des labels des colonnes 


In [27]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

### Supprimer la colonne 'RISK_MM' du jeu de données

In [28]:
df.drop(columns=['RISK_MM'], axis=1, inplace=True)

### Vérifier si la colonne est supprimée

In [29]:
assert 'RISK_MM' not in df.columns

### Afficher un summary du jeu de données

In [30]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

### Afficher la liste des noms des variables qualitatives 

In [31]:
qual = [col for col in df.columns if col not in df.describe().columns]
# df.select_dtypes(include=object, exclude=None).columns

qual

['Date',
 'Location',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday',
 'RainTomorrow']

##### Afficher le contenu de ces variables qualitatives

In [32]:
for var in qual:
    print(var, df[var].unique())

Date ['2008-12-01' '2008-12-02' '2008-12-03' ... '2008-01-29' '2008-01-30'
 '2008-01-31']
Location ['Albury' 'BadgerysCreek' 'Cobar' 'CoffsHarbour' 'Moree' 'Newcastle'
 'NorahHead' 'NorfolkIsland' 'Penrith' 'Richmond' 'Sydney' 'SydneyAirport'
 'WaggaWagga' 'Williamtown' 'Wollongong' 'Canberra' 'Tuggeranong'
 'MountGinini' 'Ballarat' 'Bendigo' 'Sale' 'MelbourneAirport' 'Melbourne'
 'Mildura' 'Nhil' 'Portland' 'Watsonia' 'Dartmoor' 'Brisbane' 'Cairns'
 'GoldCoast' 'Townsville' 'Adelaide' 'MountGambier' 'Nuriootpa' 'Woomera'
 'Albany' 'Witchcliffe' 'PearceRAAF' 'PerthAirport' 'Perth' 'SalmonGums'
 'Walpole' 'Hobart' 'Launceston' 'AliceSprings' 'Darwin' 'Katherine'
 'Uluru']
WindGustDir ['W' 'WNW' 'WSW' 'NE' 'NNW' 'N' 'NNE' 'SW' 'ENE' 'SSE' 'S' 'NW' 'SE' 'ESE'
 nan 'E' 'SSW']
WindDir9am ['W' 'NNW' 'SE' 'ENE' 'SW' 'SSE' 'S' 'NE' nan 'SSW' 'N' 'WSW' 'ESE' 'E'
 'NW' 'WNW' 'NNE']
WindDir3pm ['WNW' 'WSW' 'E' 'NW' 'W' 'SSE' 'ESE' 'ENE' 'NNW' 'SSW' 'SW' 'SE' 'N' 'S'
 'NNE' nan 'NE']
RainToday ['N

### Checker le nombre des valeurs manquantes dans chaque variable qualitative

In [33]:
df[[var for var in qual]].isnull().sum()

Date                0
Location            0
WindGustDir      9330
WindDir9am      10013
WindDir3pm       3778
RainToday        1406
RainTomorrow        0
dtype: int64

### Afficher les variables qualitatives qui contiennent des valeurs manquantes

In [34]:
df[[var for var in qual]].isnull().any()

Date            False
Location        False
WindGustDir      True
WindDir9am       True
WindDir3pm       True
RainToday        True
RainTomorrow    False
dtype: bool

### Afficher la fréquence des variables qualitatives

In [35]:
for var in qual:
    print(pd.crosstab(df[var], "freq"))

col_0       freq
Date            
2007-11-01     1
2007-11-02     1
2007-11-03     1
2007-11-04     1
2007-11-05     1
...          ...
2017-06-21    49
2017-06-22    48
2017-06-23    49
2017-06-24    48
2017-06-25    40

[3436 rows x 1 columns]
col_0             freq
Location              
Adelaide          3090
Albany            3016
Albury            3011
AliceSprings      3031
BadgerysCreek     2928
Ballarat          3028
Bendigo           3034
Brisbane          3161
Cairns            2988
Canberra          3418
Cobar             2988
CoffsHarbour      2953
Dartmoor          2943
Darwin            3192
GoldCoast         2980
Hobart            3188
Katherine         1559
Launceston        3028
Melbourne         2435
MelbourneAirport  3009
Mildura           3007
Moree             2854
MountGambier      3030
MountGinini       2907
Newcastle         2955
Nhil              1569
NorahHead         2929
NorfolkIsland     2964
Nuriootpa         3002
PearceRAAF        2762
Penrith           

### Nombre de labels: cardinalité

Le nombre de labels dans une variable qualitative est appelé la cardinalité. Un nombre élevé de labels dans une variable est appelé cardinalité élevée. Une cardinalité élevée peut poser de sérieux problèmes dans le modèle d'apprentissage automatique. Donc, Afficher la cardinalité de chaque variable qualitative.

In [36]:
df[[var for var in qual]].apply(pd.Series.nunique)

#df[[var for var in df.select_dtypes(include=object, exclude=None).columns]].apply(pd.Series.nunique)


Date            3436
Location          49
WindGustDir       16
WindDir9am        16
WindDir3pm        16
RainToday          2
RainTomorrow       2
dtype: int64

## 4. Manipulation d'une variable de type date

### Transformer la variable 'Date', qui est codé  comme strings, à une variable 'Date' de type datetime format

In [37]:
df['Date'] = pd.to_datetime(df.Date)


### Extraire l'année à partir de la variable Date et l'ajouter dans le jeu de données dans une colonne separée 'Years' 

In [38]:
df['Years'] = df.Date.dt.to_period('Y')


### Extraire le mois à partir de la variable Date et l'ajouter dans le  jeu de données dans une colonne separée 'Month' 

In [39]:
df['Month'] = df.Date.dt.month


### Extraire le jour à partir de la variable Date et l'ajouter dans le  jeu de données dans une colonne separée 'Day' 

In [127]:
df['Day'] = df.Date.dt.day


### Supprimer la variable Date du  jeu de données

In [128]:
df.drop(columns=['Date'], inplace=True)
assert 'Date' not in df.columns

In [129]:
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Years,Month,Day
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,1007.1,8.0,NaN,16.9,21.8,No,No,2008,12,1
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,1007.8,NaN,NaN,17.2,24.3,No,No,2008,12,2
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,1008.7,NaN,2.0,21.0,23.2,No,No,2008,12,3
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,1012.8,NaN,NaN,18.1,26.5,No,No,2008,12,4
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,1006.0,7.0,8.0,17.8,29.7,No,No,2008,12,5


## 5. Exploration des variables quantitatives

### Afficher la liste des variables quantitatives dans le jeu de données

### Visualiser le contenu des variables quantitatives dans le jeu de données

### Checker les valeurs manquantes dans les variables quantitatives

### Afficher le résumé statistique des variables quantitatives (describe())

## 6. Notre variable target (Y) est le RainTomorrow, Récuprer X et Y à partir du jeu de données 

## 7. Fractionnement le jeu de données en jeu d'entraînement et jeu de test (20% pour le test)

## 8. Checker les valeurs manquantes des variables quantitatives du jeu d'entraînement et de test

In [130]:
# Récupérer tout d'abord les variables quantitatives (numerical)



# Vérifier les valeurs manquantes dans la variable X_train



# Vérifier les valeurs manquantes dans la variable X_test



## 9. En utilisant SimpleImputer de Scikit-Learn remplacer les valeurs manquantes dans les variables numérique (entraînement et test) par la moyenne

## 10. Vérifier que les valeurs manquantes n'existent plus dans les variables quantitatives (entraînement et test)

## 11. En utilisant la fonction fillna() et mode() remplacer les valeurs manquantes dans les variables qualitatives par la valeur la plus fréquante pour chaque variable.

In [131]:

# Récuperer tout d'abord les variables qualitatives



## 12. Vérifier que les valeurs manquantes n'existent plus dans les variables qualitative (entraînement et test)

## 13. Encoder la variable qualitative 'RainToday' en utilisant le module category_encoders et la fonction BinaryEncoder(cols=['RainToday']) pour le jeu d'entraînement et de test.

In [132]:
# Installation: Dans Anaconda Prompt tapez la commande: pip install category_encoders
# conda install -c conda-forge category_encoders



## 14. Recréer un nouveau jeu d'entrainement en concatenant les variables numériques du jeu d'entrainement précedent avec les deux colonnes RainToday_0', 'RainToday_1'qui vont se créer à l'aide de l'étape précedente, plus les variables qualitatives transformées à des variables muettes avec la fonction get_dummies() de pandas 

## 15. Refaire l'étape précedente pour le jeu de test

## 16. Standariser votre jeu d'entrainement et de test à l'aide de StandardScaler

## 17. Créer votre modèle de régression logistique et entraînez le sur les données d'entraînement

In [133]:
# train a logistic regression model on the training set



# instantiate the model


# fit the model


## 18. Predire les résultats du modèle sur l'ensemble du test

### Calculer la probabilité que demain va pleuvoir ou pas en utilisant la fonction predict_proba sur le jeu de test

### Afficher l'accuracy score en utilisant la fonction accuracy_score sur le jeu de test

### Comparer l'accuracy du modèle sur l'ensemble d'apprentissage et l'ensemble de test, qu'est ce que vous remarquez ?


In [134]:

# Affichage du score sur les données d'apprentissage et de test



##  19. Afficher la matrice de confusion, TP, TN, FP, FN sur le jeu du test en utilisant la fonction confusion_matrix


##  20. Calculer les métriques de classification (accuracy, classification error, precision, recall, specificity) en utilisant seulement les valeurs de TP, TN, FP, FN.